In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

import time
import copy
from PIL import Image
import os

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

In [2]:
#!pip uninstall torch torchvision torchaudio -y
#!pip install torch torchvision torchaudio --upgrade

In [3]:
!pip install -U numpy scikit-learn


In [4]:
import torch
import torch.nn as nn
from torchvision import models


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. Задание классов вручную
class_names = ['Backpacks', 'Belts', 'Bra', 'Briefs', 'Capris', 'Caps', 'Casual Shoes', 'Clutches', 'Cufflinks',
               'Deodorant', 'Dresses', 'Dupatta', 'Earrings', 'Flats', 'Flip Flops', 'Formal Shoes', 'Handbags',
               'Heels', 'Innerwear Vests', 'Jackets', 'Jeans', 'Kajal and Eyeliner', 'Kurtas', 'Kurtis', 'Leggings',
               'Lip Gloss', 'Lipstick', 'Nail Polish', 'Necklace and Chains', 'Night suits', 'Nightdress', 'Pendant',
               'Perfume and Body Mist', 'Ring', 'Sandals', 'Sarees', 'Scarves', 'Shirts', 'Shorts', 'Skirts', 'Socks',
               'Sports Shoes', 'Sunglasses', 'Sweaters', 'Sweatshirts', 'Ties', 'Tops', 'Track Pants', 'Trousers',
               'Trunk', 'Tshirts', 'Tunics', 'Wallets', 'Watches']

print("Классы модели:", class_names)

# 4. Инициализация модели
model = models.resnext50_32x4d(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)


Классы модели: ['Backpacks', 'Belts', 'Bra', 'Briefs', 'Capris', 'Caps', 'Casual Shoes', 'Clutches', 'Cufflinks', 'Deodorant', 'Dresses', 'Dupatta', 'Earrings', 'Flats', 'Flip Flops', 'Formal Shoes', 'Handbags', 'Heels', 'Innerwear Vests', 'Jackets', 'Jeans', 'Kajal and Eyeliner', 'Kurtas', 'Kurtis', 'Leggings', 'Lip Gloss', 'Lipstick', 'Nail Polish', 'Necklace and Chains', 'Night suits', 'Nightdress', 'Pendant', 'Perfume and Body Mist', 'Ring', 'Sandals', 'Sarees', 'Scarves', 'Shirts', 'Shorts', 'Skirts', 'Socks', 'Sports Shoes', 'Sunglasses', 'Sweaters', 'Sweatshirts', 'Ties', 'Tops', 'Track Pants', 'Trousers', 'Trunk', 'Tshirts', 'Tunics', 'Wallets', 'Watches']


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
from google.colab import drive

# 1. Подключаем Google Drive
drive.mount('/content/drive')

# 2. Указываем путь к файлу весов на Google Drive
MODEL_PATH = '/content/drive/MyDrive/Fashion Dataset/fashion_model_weights.pt'  # Укажите ваш путь

# 3. Загружаем модель
def load_model():
    model = models.resnext50_32x4d(weights=None)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(class_names))

    # Загрузка весов с Google Drive
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        print("Модель успешно загружена с Google Drive")
    except FileNotFoundError:
        print(f"Ошибка: Файл модели не найден по пути {MODEL_PATH}")
        print("Пожалуйста, убедитесь что:")
        print("1. Google Drive подключен")
        print("2. Файл fashion_model_weights.pt находится в указанной папке")
        raise
    except Exception as e:
        print("Ошибка при загрузке модели:", e)
        raise

    model.to(device)
    model.eval()
    return model

# 4. Инициализация модели (добавьте в начало основного кода)
model = load_model()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Модель успешно загружена с Google Drive


In [6]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [7]:
model = None

def load_model():
    global model
    if model is None:
        model = models.resnext50_32x4d(weights=None)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(class_names))

        # Проверка существования файла модели
        if not os.path.exists(MODEL_PATH):
            raise FileNotFoundError(f"Файл модели не найден по пути: {MODEL_PATH}")

        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        model.to(device)
        model.eval()
    return model

# Инициализируем модель при старте
load_model()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [8]:
# 5. Загрузка весов модели
##from google.colab import files
##uploaded = files.upload()  # Выберите файл fashion_model_weights.pt
#model.load_state_dict(torch.load('fashion_model_weights.pt'))
#model.eval()  # Переводим в режим оценки


In [9]:
!pip install pyTelegramBotAPI


In [10]:
!pip install pyTelegramBotAPI

In [11]:
!pip install pyTelegramBotAPI numpy opencv-python matplotlib scikit-learn Pillow


In [12]:
!pip install torch torchvision python-telegram-bot==13.15 pillow


In [13]:
!pip install albumentations
!pip install iglovikov-helper-functions
!pip install cloths-segmentation
!pip install torch torchvision
!pip install git+https://github.com/ternaus/cloths-segmentation

  Cloning https://github.com/ternaus/cloths-segmentation to /tmp/pip-req-build-seu0vr7n
  Running command git clone --filter=blob:none --quiet https://github.com/ternaus/cloths-segmentation /tmp/pip-req-build-seu0vr7n
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/ternaus/cloths-segmentation /tmp/pip-req-build-seu0vr7n did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/ternaus/cloths-segmentation /tmp/pip-req-build-seu0vr7n did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [14]:

!pip install --upgrade scipy scikit-learn numpy

In [15]:
!pip uninstall scipy scikit-learn numpy -y
!pip install scipy scikit-learn numpy

Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
  Using cached scipy-1.15.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached scipy-1.15.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.7 MB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
ERROR: pip's dependency resolver does not currently

In [16]:
from telegram import Update
from telegram.ext import CallbackContext


In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
STYLE, SEASON, PHOTO = range(3)
# === ГРУЗИМ АННОТАЦИЮ И ИЗОБРАЖЕНИЯ ===
annotation_df = pd.read_csv("/content/drive/MyDrive/Fashion Dataset/annotation.csv")
user_filters = {}

def start(update: Update, context: CallbackContext):
    update.message.reply_text("Привет! Доступные команды:\n/find_match — найти подходящий образ по фото")

def find_match(update: Update, context: CallbackContext):
    reply_markup = ReplyKeyboardMarkup([['casual', 'business'], ['evening', 'sport']], one_time_keyboard=True)
    update.message.reply_text("Выбери стиль одежды:", reply_markup=reply_markup)
    return STYLE

def choose_style(update: Update, context: CallbackContext):
    user_filters["style"] = update.message.text
    reply_markup = ReplyKeyboardMarkup([['summer', 'autumn'], ['winter', 'spring']], one_time_keyboard=True)
    update.message.reply_text("Теперь выбери сезон:", reply_markup=reply_markup)
    return SEASON

def choose_season(update: Update, context: CallbackContext):
    user_filters["season"] = update.message.text
    update.message.reply_text("Загрузите фото вещи:")
    return PHOTO

def handle_photo(update: Update, context: CallbackContext):
    photo = update.message.photo[-1].get_file()
    image_path = "user_photo.jpg"
    photo.download(image_path)

    item_type = classify_image(image_path)
    item_color = get_dominant_color(image_path)

    style = user_filters.get("style")
    season = user_filters.get("season")

    matches = annotation_df[
        (annotation_df["style"] == style) &
        (annotation_df["weather"] == season) &
        (annotation_df["item_type"].str.lower() == item_type.lower()) &
        (annotation_df["item_color"].str.lower().str.contains(item_color.lower()))
    ]

    matched_looks = matches["image_name"].unique()[:3]  # макс 3 образа

    if len(matched_looks) == 0:
        update.message.reply_text("К сожалению, подходящих образов не найдено.")
    else:
        update.message.reply_text("Вот подходящие образы:")
        for look in matched_looks:
            file_path = f"/content/Fashion Dataset/{look}"
            update.message.bot.send_photo(chat_id=update.message.chat_id, photo=open(file_path, 'rb'))

    return ConversationHandler.END

def cancel(update: Update, context: CallbackContext):
    update.message.reply_text("Поиск отменён.")
    return ConversationHandler.END



In [19]:
#ЭТОООООООООООООО!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import cv2
import colorsys
from collections import Counter
from collections import Counter
from PIL import Image
import numpy as np
import colorsys
from matplotlib import colors

from matplotlib import colors

from telegram.ext import Updater, MessageHandler, Filters, CommandHandler

# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Классы
class_names = ['Backpacks', 'Belts', 'Bra', 'Briefs', 'Capris', 'Caps', 'Casual Shoes', 'Clutches', 'Cufflinks',
               'Deodorant', 'Dresses', 'Dupatta', 'Earrings', 'Flats', 'Flip Flops', 'Formal Shoes', 'Handbags',
               'Heels', 'Innerwear Vests', 'Jackets', 'Jeans', 'Kajal and Eyeliner', 'Kurtas', 'Kurtis', 'Leggings',
               'Lip Gloss', 'Lipstick', 'Nail Polish', 'Necklace and Chains', 'Night suits', 'Nightdress', 'Pendant',
               'Perfume and Body Mist', 'Ring', 'Sandals', 'Sarees', 'Scarves', 'Shirts', 'Shorts', 'Skirts', 'Socks',
               'Sports Shoes', 'Sunglasses', 'Sweaters', 'Sweatshirts', 'Ties', 'Tops', 'Track Pants', 'Trousers',
               'Trunk', 'Tshirts', 'Tunics', 'Wallets', 'Watches']

# Модель
#def load_model():
##    model = models.resnext50_32x4d(pretrained=False)
#    num_ftrs = model.fc.in_features
#    model.fc = nn.Linear(num_ftrs, len(class_names))
#    model.load_state_dict(torch.load("fashion_model_weights.pt", map_location=device))
#    model.to(device)
#    model.eval()
#    return model

#model = load_model()

# Трансформации
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # mean
                         [0.229, 0.224, 0.225])  # std
])

# Обработка фото
def classify_image(image_path):
    image = Image.open(image_path).convert('RGB')
    img_t = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        return class_names[preds.item()]

# Команды
# Определение цвета
def get_dominant_color(image_path, resize=100):
    """Определяет основной цвет изображения"""
    image = Image.open(image_path).convert('RGB')
    image = image.resize((resize, resize))  # Уменьшаем размер для ускорения
    pixels = list(image.getdata())

    # Удаляем белый и чёрный цвета (фон)
    pixels = [p for p in pixels if p != (255, 255, 255) and p != (0, 0, 0)]
    if not pixels:
        return "не определен"

    most_common = Counter(pixels).most_common(1)[0][0]
    hex_color = colors.to_hex([c / 255 for c in most_common])
    return get_color_name(hex_color)

def get_color_name(hex_color):
    """Присваивает читаемое название цвету по HEX"""
    rgb_color = np.array(colors.to_rgb(hex_color))
    h, s, v = colorsys.rgb_to_hsv(*rgb_color)

    if v < 0.15:
        return "чёрный"
    if v > 0.9 and s < 0.1:
        return "белый"
    if s < 0.2:
        if v < 0.5: return "тёмно-серый"
        if v < 0.8: return "серый"
        return "светло-серый"

    if h < 0.05 or h > 0.95:
        if s > 0.7:
            if v > 0.8: return "ярко-красный"
            if v > 0.5: return "красный"
            return "тёмно-красный"
        return "розовый"

    if 0.05 < h < 0.1:
        if s > 0.5:
            if v > 0.7: return "оранжевый"
            if v > 0.4: return "коричневый"
            return "тёмно-коричневый"
        return "бежевый"

    if 0.1 < h < 0.2:
        if s > 0.5:
            if v > 0.8: return "жёлтый"
            return "тёмно-жёлтый"
        return "светло-жёлтый"

    if 0.2 < h < 0.4:
        if s > 0.5:
            if v > 0.6: return "зелёный"
            return "тёмно-зелёный"
        return "светло-зелёный"

    if 0.4 < h < 0.5:
        return "бирюзовый"

    if 0.5 < h < 0.7:
        if s > 0.6:
            if v > 0.7: return "синий"
            if v > 0.4: return "тёмно-синий"
            return "очень тёмно-синий"
        return "голубой"

    if 0.7 < h < 0.8:
        if s > 0.6:
            if v > 0.7: return "фиолетовый"
            return "тёмно-фиолетовый"
        return "сиреневый"

    if 0.8 < h < 0.95:
        if s > 0.6:
            if v > 0.8: return "розовый"
            if v > 0.6: return "малиновый"
            return "бордовый"
        return "светло-розовый"

    return "неопределённый"




def start(update, context):
    update.message.reply_text("Привет! Отправь мне фото одежды, и я скажу, что это.")

def handle_photo(update, context):
    photo_file = update.message.photo[-1].get_file()
    image_path = "received.jpg"
    photo_file.download(image_path)

    # Классификация
    category = classify_image(image_path)

    # Цвет
    clothing_result = classify_image(image_path)
    color_result = get_dominant_color(image_path)

    update.message.reply_text(f"Это похоже на: {clothing_result}\nОсновной цвет: {color_result}")


# Запуск бота
#def main():
    # ЗАМЕНИ на свой токен!
    #TOKEN = "7715785252:AAHKk2U30TT8qpo6to-ZkHbFPCWpZ_naPrs"

    #updater = Updater(TOKEN, use_context=True)
    #dp = updater.dispatcher

    #dp.add_handler(CommandHandler("start", start))
    #dp.add_handler(MessageHandler(Filters.photo, handle_photo))

    #updater.start_polling()
    ##updater.idle()

#if __name__ == '__main__':
    #main()


In [20]:
import pandas as pd
import os
from telegram import InputMediaPhoto

# Путь к файлу с аннотациями
ANNOTATION_PATH = "/content/drive/MyDrive/Fashion Dataset/annotation.csv"
# Путь к папке с изображениями образов
IMAGES_BASE_PATH = "/content/drive/MyDrive/Fashion Dataset/"
MODEL_PATH = '/content/drive/MyDrive/Fashion Dataset/fashion_model_weights.pt'


def find_matching_outfits(style, season, item_type, item_color):
    try:
        # Загрузка аннотаций
        df = pd.read_csv(ANNOTATION_PATH, sep='\t')
        print("Аннотации загружены. Пример:\n", df.head())

        # Приведение к нижнему регистру
        style = style.lower()
        season = season.lower()
        item_type = item_type.lower()
        item_color = item_color.lower()

        # Поиск совпадений
        matched = df[
            (df['style'].str.lower() == style) &
            (df['weather'].str.lower() == season) &
            (df['item_type'].str.lower() == item_type) &
            (df['item_color'].str.lower() == item_color)
        ]
        print("Найдены совпадения:\n", matched)

        if matched.empty:
            print("Совпадений нет!")
            return []

        # Получаем уникальные образы
        looks = matched['image_name'].unique()
        results = []

        for look in looks[:3]:  # Не более 3 образов
            img_path = os.path.join(IMAGES_BASE_PATH, look)
            if not os.path.exists(img_path):
                print(f"Файл не найден: {img_path}")
                continue

            # Формируем описание
            items = df[df['image_name'] == look]
            desc = "Образ " + look + ": " + ", ".join(
                f"{row['item_color']} {row['item_type']}" for _, row in items.iterrows()
            )

            results.append({
                "image_path": img_path,
                "description": desc
            })

        return results

    except Exception as e:
        print(f"Ошибка в find_matching_outfits: {e}")
        return []

In [21]:
df = pd.read_csv(ANNOTATION_PATH, sep='\t')  # Убедитесь, что разделитель \t
print("Первые строки аннотаций:", df.head())

Первые строки аннотаций:   image_name,style,weather,item_type,item_color
0          look1.png,casual,summer,t-shirt,blue
1           look1.png,casual,summer,shorts,blue
2        look1.png,casual,summer,sneakers,white
3    look1.png,casual,summer,sports shoes,white
4         look1.png,casual,summer,handbag,white


In [22]:
# Дополнительные состояния
CORRECT_COLOR, CORRECT_TYPE = range(4, 6)  # После FEEDBACK

# Список цветов и типов одежды
COLORS = ["чёрный", "белый", "серый", "красный", "синий", "зелёный", "жёлтый",
          "розовый", "оранжевый", "фиолетовый", "коричневый", "бежевый"]

ITEM_TYPES = ['Backpacks', 'Belts', 'Bra', 'Briefs', 'Capris', 'Caps', 'Casual Shoes',
              'Clutches', 'Cufflinks', 'Deodorant', 'Dresses', 'Dupatta', 'Earrings',
              'Flats', 'Flip Flops', 'Formal Shoes', 'Handbags', 'Heels', 'Innerwear Vests',
              'Jackets', 'Jeans', 'Kajal and Eyeliner', 'Kurtas', 'Kurtis', 'Leggings',
              'Lip Gloss', 'Lipstick', 'Nail Polish', 'Necklace and Chains', 'Night suits',
              'Nightdress', 'Pendant', 'Perfume and Body Mist', 'Ring', 'Sandals', 'Sarees',
              'Scarves', 'Shirts', 'Shorts', 'Skirts', 'Socks', 'Sports Shoes', 'Sunglasses',
              'Sweaters', 'Sweatshirts', 'Ties', 'Tops', 'Track Pants', 'Trousers', 'Trunk',
              'Tshirts', 'Tunics', 'Wallets', 'Watches']

CORRECT_COLOR, CORRECT_TYPE = 6, 7  # Добавляем после существующих состояний
# Словарь перевода категорий (англ -> рус)
CATEGORY_TRANSLATIONS = {
    'Kurtis': 'Куртисы (женские туники)',
    'Capris': 'Капри (укороченные брюки)',
    'Flats': 'Балетки / Лоферы',
    'Heels': 'Туфли на каблуках',
    'Tops': 'Топы',
    'Tunics': 'Туники',
    'Dresses': 'Платья',
    'Kurtas': 'Курты',
    'Shorts': 'Шорты',
    'Track Pants': 'Спортивные брюки',
    'Casual Shoes': 'Повседневная обувь',
    'Jackets': 'Куртки',
    'Pendant': 'Кулон',
    'Sandals': 'Сандалии',
    'Sports Shoes': 'Кроссовки',
    'Briefs': 'Трусы',
    'Clutches': 'Клатчи',
    'Jeans': 'Джинсы',
    'Perfume and Body Mist': 'Духи',
    'Sweaters': 'Свитера',
    'Sweatshirts': 'Толстовки',
    'Trousers': 'Брюки',
    'Cufflinks': 'Запонки',
    'Earrings': 'Серьги',
    'Innerwear Vests': 'Майки',
    'Nightdress': 'Ночная сорочка',
    'Trunk': 'Боксеры',
    'Wallets': 'Кошельки',
    'Kajal and Eyeliner': 'Подводка для глаз',
    'Nail Polish': 'Лак для ногтей',
    'Skirts': 'Юбки',
    'Belts': 'Ремни',
    'Flip Flops': 'Вьетнамки',
    'Leggings': 'Леггинсы',
    'Lip Gloss': 'Блеск для губ',
    'Night suits': 'Пижамы',
    'Scarves': 'Шарфы',
    'Tshirts': 'Футболки',
    'Caps': 'Кепки',
    'Handbags': 'Сумки',
    'Lipstick': 'Помада',
    'Necklace and Chains': 'Ожерелья',
    'Ring': 'Кольцо',
    'Backpacks': 'Рюкзаки',
    'Bra': 'Бюстгальтер',
    'Formal Shoes': 'Классические туфли',
    'Shirts': 'Рубашки',
    'Socks': 'Носки',
    'Deodorant': 'Дезодорант',
    'Dupatta': 'Дупатта',
    'Sarees': 'Сари',
    'Sunglasses': 'Солнцезащитные очки',
    'Ties': 'Галстуки',
    'Watches': 'Часы'
}

RUSSIAN_ITEM_TYPES = [
    'Платья', 'Брюки', 'Рубашки', 'Футболки', 'Юбки',
    'Куртки', 'Джинсы', 'Свитера', 'Толстовки', 'Пиджаки',
    'Балетки', 'Туфли на каблуках', 'Кроссовки', 'Сандалии',
    'Сумки', 'Рюкзаки', 'Шарфы', 'Ожерелья', 'Серьги',
    'Часы', 'Кепки', 'Солнцезащитные очки', 'Топы', 'Туники',
    'Шорты', 'Леггинсы', 'Пижамы', 'Майки', 'Трусы',
    'Бюстгальтеры', 'Клатчи', 'Кошельки', 'Ремни', 'Галстуки'
]

# Обратный словарь для перевода русских названий в английские
RUSSIAN_TO_ENGLISH = {v: k for k, v in CATEGORY_TRANSLATIONS.items()}


In [23]:
from datetime import datetime
from collections import defaultdict
import csv
from collections import defaultdict

# Глобальные переменные для хранения состояния оценки
user_ratings = defaultdict(dict)

# Состояния для оценки
(RECOGNITION, ERROR_HANDLING, OUTFITS, INTERFACE, OVERALL, FINISH) = range(6)

def save_rating(user_id, data):
    with open("fashion_ratings.csv", "a", newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([
            user_id,
            data.get('recognition', ''),
            data.get('error_handling', ''),
            data.get('outfits', ''),
            data.get('interface', ''),
            data.get('overall', ''),
            datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ])

def start_rating(update, context):
    update.message.reply_text(
        "Помогите нам улучшить сервис! Оцените разные аспекты работы бота по шкале от 1 до 5:\n"
        "1 - Плохо\n"
        "5 - Отлично\n\n"
        "Оцените точность распознавания элементов одежды:"
    )
    return RECOGNITION

def recognition_rating(update, context):
    try:
        rating = int(update.message.text)
        if 1 <= rating <= 5:
            context.user_data['rating'] = {'recognition': rating}
            update.message.reply_text("Оцените обработку ошибок (насколько понятны сообщения об ошибках):")
            return ERROR_HANDLING
        else:
            update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
            return RECOGNITION
    except ValueError:
        update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
        return RECOGNITION

def error_handling_rating(update, context):
    try:
        rating = int(update.message.text)
        if 1 <= rating <= 5:
            context.user_data['rating']['error_handling'] = rating
            update.message.reply_text("Оцените подобранные образы (насколько они соответствуют вашим ожиданиям):")
            return OUTFITS
        else:
            update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
            return ERROR_HANDLING
    except ValueError:
        update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
        return ERROR_HANDLING

def outfits_rating(update, context):
    try:
        rating = int(update.message.text)
        if 1 <= rating <= 5:
            context.user_data['rating']['outfits'] = rating
            update.message.reply_text("Оцените удобство интерфейса и навигации:")
            return INTERFACE
        else:
            update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
            return OUTFITS
    except ValueError:
        update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
        return OUTFITS

def interface_rating(update, context):
    try:
        rating = int(update.message.text)
        if 1 <= rating <= 5:
            context.user_data['rating']['interface'] = rating
            update.message.reply_text("Оцените общее впечатление от сервиса:")
            return OVERALL
        else:
            update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
            return INTERFACE
    except ValueError:
        update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
        return INTERFACE

def overall_rating(update, context):
    try:
        rating = int(update.message.text)
        if 1 <= rating <= 5:
            context.user_data['rating']['overall'] = rating
            save_rating(update.message.from_user.id, context.user_data['rating'])

            # Явное завершение ConversationHandler
            context.user_data.clear()
            update.message.reply_text(
                "Спасибо за вашу оценку! Теперь можете использовать /find_match снова.",
                reply_markup=ReplyKeyboardRemove()
            )
            return ConversationHandler.END
        else:
            update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
            return OVERALL
    except ValueError:
        update.message.reply_text("Пожалуйста, введите число от 1 до 5.")
        return OVERALL

def cancel_rating(update, context):
    # Очищаем данные оценки при отмене
    if 'rating' in context.user_data:
        del context.user_data['rating']

    update.message.reply_text("Оценка отменена. Спасибо за использование нашего бота!")
    return ConversationHandler.END



In [ ]:
#ПРОБУЕМ НОВЫЙ ВАРИАНТ ДИПСИКА
import os
from telegram.ext import CallbackQueryHandler  # Добавляем этот импорт
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import cv2
import colorsys
from collections import Counter
from matplotlib import colors
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, ConversationHandler
import os
import pandas as pd
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, InputMediaPhoto
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, ConversationHandler

import os
import pandas as pd
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, InputMediaPhoto
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, ConversationHandler

import os
import random
import pandas as pd
import torch
from telegram import InlineKeyboardMarkup, InlineKeyboardButton, InputMediaPhoto
from telegram.ext import (
    Updater, CommandHandler, MessageHandler, Filters,
    ConversationHandler, CallbackQueryHandler
)
from torchvision import models

CORRECT_COLOR, CORRECT_TYPE = 4, 5

# ... (предыдущие функции: load_model, classify_image, get_dominant_color и т.д.)

def error(update, context):
    """Логирование ошибок"""
    print(f'Update {update} caused error {context.error}')
    if update and update.message:
        update.message.reply_text(
            'Произошла ошибка. Попробуйте начать заново /start',
            reply_markup=ReplyKeyboardRemove()
        )

    # Принудительная очистка
    if update and update.effective_chat:
        context.user_data.clear()


# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Состояния диалога
STYLE, SEASON, PHOTO, FEEDBACK = range(4)


def cancel(update, context):
    """Отменяет текущий диалог"""
    update.message.reply_text(
        "Поиск образов отменён. Начните заново, если хотите попробовать снова.",
        reply_markup=ReplyKeyboardRemove()
    )
    return ConversationHandler.END

# Классы одежды
class_names = ['Backpacks', 'Belts', 'Bra', 'Briefs', 'Capris', 'Caps', 'Casual Shoes', 'Clutches', 'Cufflinks',
               'Deodorant', 'Dresses', 'Dupatta', 'Earrings', 'Flats', 'Flip Flops', 'Formal Shoes', 'Handbags',
               'Heels', 'Innerwear Vests', 'Jackets', 'Jeans', 'Kajal and Eyeliner', 'Kurtas', 'Kurtis', 'Leggings',
               'Lip Gloss', 'Lipstick', 'Nail Polish', 'Necklace and Chains', 'Night suits', 'Nightdress', 'Pendant',
               'Perfume and Body Mist', 'Ring', 'Sandals', 'Sarees', 'Scarves', 'Shirts', 'Shorts', 'Skirts', 'Socks',
               'Sports Shoes', 'Sunglasses', 'Sweaters', 'Sweatshirts', 'Ties', 'Tops', 'Track Pants', 'Trousers',
               'Trunk', 'Tshirts', 'Tunics', 'Wallets', 'Watches']

# Стили одежды
styles = ["будничный", "деловой", "вечерний", "спортивный"]

# Сезоны
seasons = ["лето", "осень", "зима", "весна"]

# Модель классификации
def load_model():
    model = models.resnext50_32x4d(weights=None)  # Используем новый синтаксис
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, len(class_names))
    model.load_state_dict(torch.load("fashion_model_weights.pt", map_location=device))
    model.to(device)
    model.eval()
    return model

# Трансформации изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Классификация изображения
def classify_image(image_path):
    """Возвращает английское название категории (как в аннотациях)"""
    image = Image.open(image_path).convert('RGB')
    img_t = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        return class_names[preds.item()]  # Возвращаем английское название

# Определение доминирующего цвета
def get_dominant_color(image_path, resize=100):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((resize, resize))
    pixels = list(image.getdata())
    pixels = [p for p in pixels if p != (255, 255, 255) and p != (0, 0, 0)]
    if not pixels:
        return "не определен"
    most_common = Counter(pixels).most_common(1)[0][0]
    hex_color = colors.to_hex([c / 255 for c in most_common])
    return get_color_name(hex_color)

def get_color_name(hex_color):
    rgb_color = np.array(colors.to_rgb(hex_color))
    h, s, v = colorsys.rgb_to_hsv(*rgb_color)

    if v < 0.15:
        return "чёрный"
    if v > 0.9 and s < 0.1:
        return "белый"
    if s < 0.2:
        if v < 0.5: return "тёмно-серый"
        if v < 0.8: return "серый"
        return "светло-серый"

    if h < 0.05 or h > 0.95:
        if s > 0.7:
            if v > 0.8: return "ярко-красный"
            if v > 0.5: return "красный"
            return "тёмно-красный"
        return "розовый"

    if 0.05 < h < 0.1:
        if s > 0.5:
            if v > 0.7: return "оранжевый"
            if v > 0.4: return "коричневый"
            return "тёмно-коричневый"
        return "бежевый"

    if 0.1 < h < 0.2:
        if s > 0.5:
            if v > 0.8: return "жёлтый"
            return "тёмно-жёлтый"
        return "светло-жёлтый"

    if 0.2 < h < 0.4:
        if s > 0.5:
            if v > 0.6: return "зелёный"
            return "тёмно-зелёный"
        return "светло-зелёный"

    if 0.4 < h < 0.5:
        return "бирюзовый"

    if 0.5 < h < 0.7:
        if s > 0.6:
            if v > 0.7: return "синий"
            if v > 0.4: return "тёмно-синий"
            return "очень тёмно-синий"
        return "голубой"

    if 0.7 < h < 0.8:
        if s > 0.6:
            if v > 0.7: return "фиолетовый"
            return "тёмно-фиолетовый"
        return "сиреневый"

    if 0.8 < h < 0.95:
        if s > 0.6:
            if v > 0.8: return "розовый"
            if v > 0.6: return "малиновый"
            return "бордовый"
        return "светло-розовый"

    return "неопределённый"

def type_selected(update, context):
    query = update.callback_query
    query.answer()

    # Получаем выбранный тип (английское название из callback_data)
    selected_type = query.data.replace('type_', '')

    # Сохраняем английское название в user_data
    context.user_data['corrected_type'] = selected_type

    # Получаем русское название для отображения
    russian_type = CATEGORY_TRANSLATIONS.get(selected_type, selected_type)

    # Получаем цвет (исправленный или оригинальный)
    color = context.user_data.get('corrected_color', context.user_data.get('item_color', 'не определен'))

    # Обновляем параметры
    context.user_data['item_type'] = selected_typeЫ
    context.user_data['item_color'] = color

    # Ищем образы с новыми параметрами
    outfits = find_matching_outfits(
        context.user_data['style'],
        context.user_data['season'],
        selected_type,
        color
    )

    if outfits:
        context.user_data['all_outfits'] = random.sample(outfits, len(outfits))
        context.user_data['current_outfit_idx'] = 0
        return show_next_outfit(update, context)
    else:
        # Если образы не найдены, показываем параметры и предлагаем изменить
        keyboard = [
            [InlineKeyboardButton("Изменить цвет", callback_data='correct_color')],
            [InlineKeyboardButton("Изменить тип одежды", callback_data='correct_type')],
            [InlineKeyboardButton("Изменить оба параметра", callback_data='correct_both')],
            [InlineKeyboardButton("Начать заново", callback_data='restart')]
        ]

        query.edit_message_text(
            f"😕 Не найдено образов с параметрами:\n"
            f"Стиль: {context.user_data['style']}\n"
            f"Сезон: {context.user_data['season']}\n"
            f"Тип: {russian_type}\n"
            f"Цвет: {color}\n\n"
            f"Что хотите изменить?",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return FEEDBACK



def create_type_buttons():
    buttons = []
    row = []

    for i, russian_type in enumerate(RUSSIAN_ITEM_TYPES):
        # Получаем английское название для callback_data
        eng_type = RUSSIAN_TO_ENGLISH.get(russian_type, russian_type)
        row.append(InlineKeyboardButton(russian_type, callback_data=f'type_{eng_type}'))

        # 3 кнопки в ряд
        if (i + 1) % 3 == 0:
            buttons.append(row)
            row = []

    if row:  # Добавляем последний неполный ряд
        buttons.append(row)

    return buttons


def feedback_handler(update, context):
    query = update.callback_query
    query.answer()

    if query.data == 'like_yes':
        query.edit_message_text("✅ Отлично! Сохранили ваш выбор. Попробуйте еще раз /find_match")
        return ConversationHandler.END
    elif query.data == 'like_no':
        return show_next_outfit(update, context)
    elif query.data == 'done':
        query.edit_message_text("🔚 Показаны все варианты. Начните заново /find_match")
        return ConversationHandler.END
    elif query.data == 'restart':
        query.edit_message_text("🔄 Начинаем заново. Используйте /find_match")
        return ConversationHandler.END
    elif query.data == 'change_params':
        current_type = context.user_data.get('item_type', 'не определен')
        translated_type = CATEGORY_TRANSLATIONS.get(current_type, current_type)

        keyboard = [
            [InlineKeyboardButton("Изменить цвет", callback_data='correct_color')],
            [InlineKeyboardButton("Изменить тип одежды", callback_data='correct_type')],
            [InlineKeyboardButton("Изменить оба параметра", callback_data='correct_both')],
            [InlineKeyboardButton("Начать заново", callback_data='restart')]
        ]
        query.edit_message_text(
            f"Текущая классификация:\n"
            f"Тип: {translated_type}\n"
            f"Цвет: {context.user_data.get('item_color', 'не определен')}\n\n"
            "Что именно неправильно определено?",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return FEEDBACK
    elif query.data.startswith('correct_'):
        action = query.data.replace('correct_', '')

        if action == 'color':
            context.user_data.pop('correcting_both', None)

            buttons = []
            row = []
            for i, color in enumerate(COLORS):
                row.append(InlineKeyboardButton(color, callback_data=f'color_{color}'))
                if (i + 1) % 3 == 0:
                    buttons.append(row)
                    row = []
            if row:
                buttons.append(row)

            reply_markup = InlineKeyboardMarkup(buttons)

            query.edit_message_text(
                "Выберите правильный цвет одежды:",
                reply_markup=reply_markup
            )
            return CORRECT_COLOR

        elif action == 'type':
            context.user_data.pop('correcting_both', None)

            # Создаем кнопки с русскими названиями
            buttons = []
            row = []
            for i, russian_type in enumerate(RUSSIAN_ITEM_TYPES):
                eng_type = RUSSIAN_TO_ENGLISH.get(russian_type, russian_type)
                row.append(InlineKeyboardButton(russian_type, callback_data=f'type_{eng_type}'))
                if (i + 1) % 3 == 0:
                    buttons.append(row)
                    row = []
            if row:
                buttons.append(row)

            reply_markup = InlineKeyboardMarkup(buttons)

            query.edit_message_text(
                "Выберите правильный тип одежды:",
                reply_markup=reply_markup
            )
            return CORRECT_TYPE

        elif action == 'both':
            context.user_data['correcting_both'] = True

            buttons = []
            row = []
            for i, color in enumerate(COLORS):
                row.append(InlineKeyboardButton(color, callback_data=f'color_{color}'))
                if (i + 1) % 3 == 0:
                    buttons.append(row)
                    row = []
            if row:
                buttons.append(row)

            reply_markup = InlineKeyboardMarkup(buttons)

            query.edit_message_text(
                "Сначала выберите правильный цвет одежды:",
                reply_markup=reply_markup
            )
            return CORRECT_COLOR







# Поиск подходящих образов (заглушка - нужно реализовать вашу логику)
def find_matching_outfits(style, season, item_type, item_color):
    try:
        df = pd.read_csv(ANNOTATION_PATH, sep=',')

        # Преобразование терминов
        style_map = {"будничный": "casual", "деловой": "formal", "вечерний": "evening", "спортивный": "sport"}
        season_map = {"лето": "summer", "осень": "autumn", "зима": "winter", "весна": "spring"}
        type_map = {
    # Основные категории
    "платье": "Dresses",
    "брюки": "Trousers",
    "рубашка": "Shirts",
    "футболка": "Tshirts",
    "юбка": "Skirts",
    "куртка": "Jackets",
    "джинсы": "Jeans",
    "свитер": "Sweaters",
    "кофта": "Sweatshirts",
    "пиджак": "Jackets",
    "обувь": "Casual Shoes",
    "аксессуар": "Accessories",
    "верхняя одежда": "Jackets",

    # Все остальные классы из class_names
    "рюкзак": "Backpacks",
    "ремень": "Belts",
    "бюстгальтер": "Bra",
    "трусы": "Briefs",
    "капри": "Capris",
    "кепка": "Caps",
    "повседневная обувь": "Casual Shoes",
    "клатч": "Clutches",
    "запонки": "Cufflinks",
    "дезодорант": "Deodorant",
    "дупатта": "Dupatta",
    "серьги": "Earrings",
    "балетки": "Flats",
    "вьетнамки": "Flip Flops",
    "формальная обувь": "Formal Shoes",
    "сумка": "Handbags",
    "каблуки": "Heels",
    "майка": "Innerwear Vests",
    "подводка для глаз": "Kajal and Eyeliner",
    "курта": "Kurtas",
    "курти": "Kurtis",
    "леггинсы": "Leggings",
    "блеск для губ": "Lip Gloss",
    "помада": "Lipstick",
    "лак для ногтей": "Nail Polish",
    "ожерелье": "Necklace and Chains",
    "пижама": "Night suits",
    "ночная сорочка": "Nightdress",
    "кулон": "Pendant",
    "парфюм": "Perfume and Body Mist",
    "кольцо": "Ring",
    "сандалии": "Sandals",
    "сари": "Sarees",
    "шарф": "Scarves",
    "шорты": "Shorts",
    "носки": "Socks",
    "спортивная обувь": "Sports Shoes",
    "солнцезащитные очки": "Sunglasses",
    "галстук": "Ties",
    "топ": "Tops",
    "спортивные штаны": "Track Pants",
    "трусы-боксеры": "Trunk",
    "туника": "Tunics",
    "кошелек": "Wallets",
    "часы": "Watches",

    # Дополнительные варианты названий
    "спортивный костюм": "Track Pants",
    "верх": "Tops",
    "низ": "Trousers",
    "верхняя": "Jackets",
    "нижнее белье": "Briefs",
    "украшения": "Earrings",
    "косметика": "Lipstick",
    "сумки": "Handbags",
    "обувь": "Casual Shoes"
}

        style_en = style_map.get(style.lower(), style.lower())
        season_en = season_map.get(season.lower(), season.lower())
        type_en = type_map.get(item_type.lower(), item_type)

        # Находим все подходящие образы
        matched = df[
            (df['style'].str.lower() == style_en) &
            (df['weather'].str.lower() == season_en) &
            (df['item_type'].str.lower() == type_en.lower()) &
            (df['item_color'].str.lower() == item_color.lower())
        ]

        if matched.empty:
            return []

        # Возвращаем полные пути к изображениям
        return [os.path.join(IMAGES_BASE_PATH, img) for img in matched['image_name'].unique()]

    except Exception as e:
        print(f"Ошибка поиска образов: {e}")
        return []



def show_next_outfit(update, context):
    try:
        if 'all_outfits' not in context.user_data or context.user_data['current_outfit_idx'] >= len(context.user_data['all_outfits']):
            update.message.reply_text("🏁 Все возможные образы показаны. Попробуйте еще раз /find_match.")
            return ConversationHandler.END

        outfit_path = context.user_data['all_outfits'][context.user_data['current_outfit_idx']]
        context.user_data['current_outfit_idx'] += 1
        outfit_name = os.path.basename(outfit_path)

        try:
            df = pd.read_csv(ANNOTATION_PATH, sep=',')
            outfit_data = df[df['image_name'] == outfit_name]

            # Формируем описание с русскими названиями
            desc_parts = ["✨ Образ:"]
            for _, row in outfit_data.iterrows():
                item_type = CATEGORY_TRANSLATIONS.get(row['item_type'], row['item_type'])
                desc_parts.append(f"- {row['item_color']} {item_type}")

            desc = "\n".join(desc_parts)

        except Exception as e:
            print(f"Ошибка чтения аннотаций: {e}")
            desc = "✨ Образ (описание недоступно)"

        # Отправка фото и описания
        try:
            with open(outfit_path, 'rb') as photo_file:
                if update.callback_query:
                    update.callback_query.message.reply_photo(photo_file, caption=desc)
                else:
                    update.message.reply_photo(photo_file, caption=desc)
        except FileNotFoundError:
            if update.callback_query:
                update.callback_query.message.reply_text(desc + "\n\n⚠️ Изображение не найдено")
            else:
                update.message.reply_text(desc + "\n\n⚠️ Изображение не найдено")

        # Клавиатура с действиями
        remaining = len(context.user_data['all_outfits']) - context.user_data['current_outfit_idx']
        keyboard = [
            [
                InlineKeyboardButton("👍 Нравится", callback_data='like_yes'),
                InlineKeyboardButton("👎 Не нравится", callback_data='like_no')
            ] if remaining > 0 else [
                InlineKeyboardButton("👍 Нравится", callback_data='like_yes'),
                InlineKeyboardButton("🏁 Завершить", callback_data='done')
            ],
            [InlineKeyboardButton("❌ Неправильная классификация", callback_data='change_params')]
        ]

        reply_markup = InlineKeyboardMarkup(keyboard)
        target = update.callback_query.message if update.callback_query else update.message
        target.reply_text(
            f"Осталось вариантов: {remaining}\nВыберите действие:",
            reply_markup=reply_markup
        )

        return FEEDBACK

    except Exception as e:
        print(f"Ошибка в show_next_outfit: {e}")
        error_msg = "⚠️ Ошибка при показе образов. Попробуйте ещё раз /find_match"
        if update.callback_query:
            update.callback_query.message.reply_text(error_msg)
        else:
            update.message.reply_text(error_msg)
        return ConversationHandler.END

#

def color_selected(update, context):
    query = update.callback_query
    query.answer()

    # Сохраняем выбранный цвет
    selected_color = query.data.replace('color_', '')
    context.user_data['corrected_color'] = selected_color

    # Проверяем, нужно ли также выбирать тип (коррекция обоих параметров)
    if context.user_data.get('correcting_both', False):
        # Удаляем временный флаг
        del context.user_data['correcting_both']

        # Создаем кнопки для выбора типа одежды
        buttons = []
        row = []
        for i, item_type in enumerate(ITEM_TYPES):
            row.append(InlineKeyboardButton(item_type, callback_data=f'type_{item_type}'))
            if (i + 1) % 2 == 0:  # 2 кнопки в строке
                buttons.append(row)
                row = []
        if row:  # Добавляем последнюю неполную строку
            buttons.append(row)

        reply_markup = InlineKeyboardMarkup(buttons)

        query.edit_message_text(
            f"Выбран цвет: {selected_color}\nТеперь выберите тип одежды:",
            reply_markup=reply_markup
        )
        return CORRECT_TYPE
    else:
        # Используем оригинальный тип и новый цвет
        current_type = context.user_data.get('item_type', 'не определен')
        current_color = context.user_data.get('item_color', 'не определен')

        # Обновляем цвет в user_data
        context.user_data['item_color'] = selected_color

        # Ищем образы с исправленными параметрами
        outfits = find_matching_outfits(
            context.user_data['style'],
            context.user_data['season'],
            current_type,
            selected_color
        )

        if outfits:
            # Сохраняем новые образы и сбрасываем индекс
            context.user_data['all_outfits'] = random.sample(outfits, len(outfits))
            context.user_data['current_outfit_idx'] = 0

            # Показываем первый образ из новых
            return show_next_outfit(update, context)
        else:
            # Если образы не найдены, показываем текущую классификацию и предлагаем исправить
            keyboard = [
                [InlineKeyboardButton("Изменить цвет", callback_data='correct_color')],
                [InlineKeyboardButton("Изменить тип одежды", callback_data='correct_type')],
                [InlineKeyboardButton("Изменить оба параметра", callback_data='correct_both')],
                [InlineKeyboardButton("Начать заново", callback_data='restart')]
            ]

            query.edit_message_text(
                f"😕 Не найдено образов с параметрами:\n"
                f"Стиль: {context.user_data['style']}\n"
                f"Сезон: {context.user_data['season']}\n"
                f"Тип: {current_type}\n"
                f"Цвет: {selected_color}\n\n"
                f"Что хотите изменить?",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return FEEDBACK



def type_selected(update, context):
    query = update.callback_query
    query.answer()

    # Сохраняем выбранный тип
    selected_type = query.data.replace('type_', '')
    context.user_data['corrected_type'] = selected_type

    # Получаем цвет (исправленный или оригинальный)
    color = context.user_data.get('corrected_color', context.user_data.get('item_color', 'не определен'))

    # Обновляем параметры в user_data
    context.user_data['item_type'] = selected_type
    context.user_data['item_color'] = color

    # Ищем образы с исправленными параметрами
    outfits = find_matching_outfits(
        context.user_data['style'],
        context.user_data['season'],
        selected_type,
        color
    )

    if outfits:
        # Сохраняем новые образы и сбрасываем индекс
        context.user_data['all_outfits'] = random.sample(outfits, len(outfits))
        context.user_data['current_outfit_idx'] = 0

        # Показываем первый образ из новых
        return show_next_outfit(update, context)
    else:
        # Если образы не найдены, показываем текущую классификацию и предлагаем исправить
        keyboard = [
            [InlineKeyboardButton("Изменить цвет", callback_data='correct_color')],
            [InlineKeyboardButton("Изменить тип одежды", callback_data='correct_type')],
            [InlineKeyboardButton("Изменить оба параметра", callback_data='correct_both')],
            [InlineKeyboardButton("Начать заново", callback_data='restart')]
        ]

        query.edit_message_text(
            f"😕 Не найдено образов с параметрами:\n"
            f"Стиль: {context.user_data['style']}\n"
            f"Сезон: {context.user_data['season']}\n"
            f"Тип: {selected_type}\n"
            f"Цвет: {color}\n\n"
            f"Что хотите изменить?",
            reply_markup=InlineKeyboardMarkup(keyboard)
        )
        return FEEDBACK


def start(update, context):
    commands = [
        ['/find_match - Найти образ'],
        ['/help - Помощь']
    ]
    reply_markup = ReplyKeyboardMarkup(commands, resize_keyboard=True)
    update.message.reply_text(
        "Привет! Я бот для подбора образов одежды.\n"
        "Используй команды ниже:",
        reply_markup=reply_markup
    )

def help_command(update, context):
    update.message.reply_text(
        "Доступные команды:\n"
        "/find_match - подобрать образ по вашей одежде\n"
        "/help - показать это сообщение"
    )

def find_match(update, context):
    # Принудительно завершаем любые активные ConversationHandler
    context.user_data.clear()  # Полная очистка user_data

    # Добавляем лог для отладки
    print("DEBUG: find_match вызван, user_data очищен")

    style_options = [["будничный", "деловой"], ["вечерний", "спортивный"]]
    reply_markup = ReplyKeyboardMarkup(style_options, resize_keyboard=True)
    update.message.reply_text(
        "Выберите стиль одежды:",
        reply_markup=reply_markup
    )
    return STYLE



def style_selected(update, context):
    user_style = update.message.text.lower()
    if user_style not in styles:
        update.message.reply_text("Пожалуйста, выберите стиль из предложенных вариантов.")
        return STYLE

    context.user_data['style'] = user_style

    season_options = [["лето", "осень"], ["зима", "весна"]]
    reply_markup = ReplyKeyboardMarkup(season_options, resize_keyboard=True)
    update.message.reply_text(
        "Теперь выберите сезон:",
        reply_markup=reply_markup
    )
    return SEASON

def season_selected(update, context):
    user_season = update.message.text.lower()
    if user_season not in seasons:
        update.message.reply_text("Пожалуйста, выберите сезон из предложенных вариантов.")
        return SEASON

    context.user_data['season'] = user_season
    update.message.reply_text(
        "Теперь загрузите фото одного элемента одежды:",
        reply_markup=ReplyKeyboardRemove()
    )
    return PHOTO

from telegram import InlineKeyboardMarkup, InlineKeyboardButton


def cancel_rating(update, context):
    update.message.reply_text("Оценка отменена. Можете продолжить использование бота.")
    return ConversationHandler.END

rating_conv_handler = ConversationHandler(
    entry_points=[CommandHandler('rate', start_rating)],
    states={
        RECOGNITION: [MessageHandler(Filters.text & ~Filters.command, recognition_rating)],
        ERROR_HANDLING: [MessageHandler(Filters.text & ~Filters.command, error_handling_rating)],
        OUTFITS: [MessageHandler(Filters.text & ~Filters.command, outfits_rating)],
        INTERFACE: [MessageHandler(Filters.text & ~Filters.command, interface_rating)],
        OVERALL: [MessageHandler(Filters.text & ~Filters.command, overall_rating)],
    },
    fallbacks=[CommandHandler('cancel', cancel_rating)],
    per_message=False  # Добавьте этот параметр
)

def photo_received(update, context):
    try:
        # Скачиваем фото
        photo_file = update.message.photo[-1].get_file()
        image_path = "temp_photo.jpg"
        photo_file.download(image_path)

        # Классификация одежды и цвета
        try:
            item_type = classify_image(image_path)
            item_color = get_dominant_color(image_path)

            # Сохраняем параметры
            context.user_data.update({
                'original_type': item_type,
                'original_color': item_color,
                'item_type': item_type,
                'item_color': item_color,
                'all_outfits': [],
                'current_outfit_idx': 0
            })

            # Отправляем пользователю результат классификации
            update.message.reply_text(
                f"🔍 Результаты анализа:\n"
                f"Тип одежды: {item_type}\n"
                f"Цвет: {item_color}\n\n"
                f"Ищем подходящие образы..."
            )

        except Exception as e:
            print(f"Ошибка классификации: {e}")
            update.message.reply_text("⚠️ Ошибка анализа изображения. Попробуйте другое фото.")
            return ConversationHandler.END

        # Поиск подходящих образов
        all_outfits = find_matching_outfits(
            context.user_data['style'],
            context.user_data['season'],
            item_type,
            item_color
        )

        if not all_outfits:
            # Если образы не найдены - предлагаем изменить параметры
            keyboard = [
                [InlineKeyboardButton("Изменить цвет", callback_data='correct_color')],
                [InlineKeyboardButton("Изменить тип одежды", callback_data='correct_type')],
                [InlineKeyboardButton("Изменить оба параметра", callback_data='correct_both')],
                [InlineKeyboardButton("Начать заново", callback_data='restart')]
            ]

            update.message.reply_text(
                f"😕 Не найдено образов с параметрами:\n"
                f"Стиль: {context.user_data['style']}\n"
                f"Сезон: {context.user_data['season']}\n"
                f"Тип: {item_type}\n"
                f"Цвет: {item_color}\n\n"
                f"Что хотите изменить?",
                reply_markup=InlineKeyboardMarkup(keyboard)
            )
            return FEEDBACK

        # Сохраняем найденные образы
        context.user_data['all_outfits'] = all_outfits
        context.user_data['current_outfit_idx'] = 0

        # Показываем первый образ
        return show_next_outfit(update, context)

    except Exception as e:
        print(f"Ошибка в photo_received: {e}")
        update.message.reply_text(
            "⚠️ Произошла ошибка при обработке фото. Попробуйте ещё раз /find_match",
            reply_markup=ReplyKeyboardRemove()
        )
        return ConversationHandler.END

    finally:
        # Удаляем временный файл
        if os.path.exists(image_path):
            os.remove(image_path)

FEEDBACK = 3  # Добавляем после существующих состояний
def main():
    # Проверка доступности файлов перед запуском
    if not os.path.exists(MODEL_PATH):
        print(f"ОШИБКА: Файл модели не найден по пути {MODEL_PATH}")
        return

    if not os.path.exists(ANNOTATION_PATH):
        print(f"ОШИБКА: Файл аннотаций не найден по пути {ANNOTATION_PATH}")
        return

    TOKEN = "7715785252:AAHKk2U30TT8qpo6to-ZkHbFPCWpZ_naPrs"  # Замените на ваш токен

    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    # Обработчик диалога
    # Измените параметр per_message на False
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('find_match', find_match)],
        states={
            STYLE: [MessageHandler(Filters.text & ~Filters.command, style_selected)],
            SEASON: [MessageHandler(Filters.text & ~Filters.command, season_selected)],
            PHOTO: [MessageHandler(Filters.photo & ~Filters.command, photo_received)],
            FEEDBACK: [CallbackQueryHandler(feedback_handler)],
            CORRECT_COLOR: [CallbackQueryHandler(color_selected)],
            CORRECT_TYPE: [CallbackQueryHandler(type_selected)]
        },
        fallbacks=[CommandHandler('cancel', cancel)],
        per_message=False
    )

    rating_conv_handler = ConversationHandler(
        entry_points=[CommandHandler('rate', start_rating)],
        states={
            RECOGNITION: [MessageHandler(Filters.text & ~Filters.command, recognition_rating)],
            ERROR_HANDLING: [MessageHandler(Filters.text & ~Filters.command, error_handling_rating)],
            OUTFITS: [MessageHandler(Filters.text & ~Filters.command, outfits_rating)],
            INTERFACE: [MessageHandler(Filters.text & ~Filters.command, interface_rating)],
            OVERALL: [MessageHandler(Filters.text & ~Filters.command, overall_rating)],
        },
        fallbacks=[CommandHandler('cancel', cancel_rating)],
        per_message=False,  # Важно!
        allow_reentry=True  # Разрешает повторный вход
    )

    dp.add_handler(conv_handler)
    dp.add_handler(rating_conv_handler)

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))

    dp.add_error_handler(error)


    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/telegram/ext/conversationhandler.py:288: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn(


DEBUG: find_match вызван, user_data очищен
DEBUG: find_match вызван, user_data очищен
DEBUG: find_match вызван, user_data очищен
